In [2]:
#spark.stop()

In [3]:
import os, sys
import datetime as dt
DEVEL_SRC = os.path.join(os.environ.get('BDA_USER_HOME', ''), "src", "devel")
if DEVEL_SRC not in sys.path:
    sys.path.append(DEVEL_SRC)

USECASES_SRC = os.path.join(DEVEL_SRC, "use-cases") # TODO when - is removed, remove also this line and adapt imports
if USECASES_SRC not in sys.path: 
    sys.path.append(USECASES_SRC)
    
AMDOCS_SRC = os.path.join(DEVEL_SRC, "amdocs_informational_dataset") # TODO when - is removed, remove also this line and adapt imports
if AMDOCS_SRC not in sys.path: 
    sys.path.append(AMDOCS_SRC)
    
import pykhaos.utils.custom_logger as clogger
logging_file = os.path.join(os.environ.get('BDA_USER_HOME', ''), "logging",
                                    "out_" + dt.datetime.now().strftime("%Y%m%d_%H%M%S") + ".log")
logger = clogger.configure_logger(log_filename=logging_file, std_channel=sys.stderr, logger_name="")
logger.info("Logging to file {}".format(logging_file))    
    
    
from project.project_generic import Project


import pykhaos.utils.notebooks as nb

project_obj = Project("encuestas bajas", "encuestas bajas")   

RUNNING_FROM_NOTEBOOK = nb.isnotebook()
import matplotlib.pyplot as plt
if RUNNING_FROM_NOTEBOOK:
    %load_ext autoreload
    %autoreload 2
    %matplotlib inline  
    
#logger = my_project.logger

if not RUNNING_FROM_NOTEBOOK:
    args = my_project.arg_parser()

_initialize spark
Ended spark session: 37.8607189655 secs | default parallelism=2


In [4]:
spark=project_obj.spark

In [5]:
%load_ext autoreload
%autoreload 2
%matplotlib inline 
from churn.datapreparation.data_loader import get_port_requests_table
from churn.datapreparation.config import Config

config_obj = Config(None)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/var/SP/data/home/csanc109/src/devel/use-cases/churn
Reading config from file /var/SP/data/home/csanc109/src/devel/use-cases/churn/input/datapreparation_churn.yaml
Reading internal config from file /var/SP/data/home/csanc109/src/devel/use-cases/churn/datapreparation/config/internal_config.yaml


IOError: [Errno 2] No such file or directory: '/var/SP/data/home/csanc109/src/devel/use-cases/churn/datapreparation/config/internal_config.yaml'

# Encuestas Bajas

In [34]:
from churn.datapreparation.engine.bajas_data_loader import load_reasons_encuestas
df_bajas = load_reasons_encuestas(spark, config_obj=None)



Returning all information churn_reason...


In [35]:
yearmonth_list=df_bajas.select('yearmonth').distinct().rdd.map(lambda r: r[0]).collect()

# Read complete car

In [26]:
from amdocs_informational_dataset.engine.call_centre_calls import CallCentreCalls
from pyspark.sql.functions import collect_set, concat, size, coalesce, col, lpad, struct, count as sql_count, regexp_replace, lit, min as sql_min, max as sql_max, collect_list, udf, when, desc, row_number
from pyspark.sql.types import StringType, ArrayType, MapType, StructType, StructField, IntegerType
from pyspark.sql.window import Window



In [45]:
reasons_list=df_bajas.select('Net_Motivo_Principal').distinct().rdd.map(lambda r: r[0]).collect()
reasons_list

[u'Cambio de situaci\ufffdn personal',
 u'Precio-Necesidad de ahorro',
 u'Problemas con el servicio',
 u'No disponibilidad de Fibra',
 u'Mala atenci\ufffdn al cliente',
 u'Enga\ufffdo comercial',
 u'Falta de contenidos de TV',
 u'Otros',
 u'Problemas con factura']

In [51]:
from churn.utils.general_functions import get_nif_msisdn

def get_complete_info_encuestas(): #FIXME
    df_old_0531 = get_nif_msisdn(spark, "20180531").withColumn("closing_day", lit("20180531"))
    df_old_0630 = get_nif_msisdn(spark, "20180630").withColumn("closing_day", lit("20180630"))
    df_old_0731 = get_nif_msisdn(spark, "20180731").withColumn("closing_day", lit("20180731"))
    df_old_0831 = get_nif_msisdn(spark, "20180831").withColumn("closing_day", lit("20180831"))
    df_old_0930 = get_nif_msisdn(spark, "20180930").withColumn("closing_day", lit("20180930"))
    df_old_1031 = get_nif_msisdn(spark, "20181031").withColumn("closing_day", lit("20181031"))

    cols = list(set(df_old_0531.columns) & set(df_old_0630.columns) & set(df_old_0731.columns) & set(df_old_0831.columns) & set(df_old_0930.columns) & set(df_old_1031.columns))

    def union_all(dfs):
        if len(dfs) > 1:
            return dfs[0].unionAll(union_all(dfs[1:]))
        else:
            return dfs[0]

    df_car_joined = union_all([df_old_0531.select(cols), df_old_0630.select(cols), df_old_0731.select(cols), df_old_0831.select(cols), df_old_0930.select(cols), df_old_1031.select(cols)])
    w_order = Window().partitionBy("nif_cliente").orderBy(desc("closing_day"))
    # Filter the repeteated rows, keeping the last one
    df_car_joined = (df_car_joined.withColumn("rowNumorder", row_number().over(w_order)).where(col('rowNumorder') == 1)).drop("rowNumorder")
    df_complete = df_bajas.join(df_car_joined, on=df_bajas["NIF"]==df_car_joined["NIF_CLIENTE"], how="left").drop("NIF_CLIENTE", 'service_processed_at',
     'service_file_id')
    
    return df_complete


REASONS_MAPPING = {u'Cambio de situaci\ufffdn personal' : "SITUACION", 
                   u'Precio-Necesidad de ahorro' : "PRECIO",
                   u'Problemas con el servicio': "SERVICIO/ATENCION",
                   u'No disponibilidad de Fibra' : "NO_FIBRA",
                   u'Mala atenci\ufffdn al cliente' : "SERVICIO/ATENCION",
                   u'Enga\ufffdo comercial' : "FRAUD",
                   u'Falta de contenidos de TV' : "CONTENIDOS",
                   u'Otros' : "OTROS",
                   u'Problemas con factura' : "BILLING"}


from pyspark.sql.functions import col, create_map, lit
from itertools import chain

mapping_expr = create_map([lit(x) for x in chain(*REASONS_MAPPING.items())])

df_complete = get_complete_info_encuestas()
df_complete = df_complete.withColumn("REASON", mapping_expr.getItem(col('Net_Motivo_Principal')))



('amdocs_table_reader', 'customer', '20180531')
Loading /data/udf/vf_es/amdocs_inf_dataset/customer/year=2018/month=5/day=31
('amdocs_table_reader', 'service', '20180531')
Loading /data/udf/vf_es/amdocs_inf_dataset/service/year=2018/month=5/day=31
('amdocs_table_reader', 'customer', '20180630')
Loading /data/udf/vf_es/amdocs_inf_dataset/customer/year=2018/month=6/day=30
('amdocs_table_reader', 'service', '20180630')
Loading /data/udf/vf_es/amdocs_inf_dataset/service/year=2018/month=6/day=30
('amdocs_table_reader', 'customer', '20180731')
Loading /data/udf/vf_es/amdocs_inf_dataset/customer/year=2018/month=7/day=31
('amdocs_table_reader', 'service', '20180731')
Loading /data/udf/vf_es/amdocs_inf_dataset/service/year=2018/month=7/day=31
('amdocs_table_reader', 'customer', '20180831')
Loading /data/udf/vf_es/amdocs_inf_dataset/customer/year=2018/month=8/day=31
('amdocs_table_reader', 'service', '20180831')
Loading /data/udf/vf_es/amdocs_inf_dataset/service/year=2018/month=8/day=31
('amdocs

In [53]:
df_complete.select('REASON').distinct().rdd.map(lambda r: r[0]).collect()

[u'FRAUD',
 u'BILLING',
 u'SERVICIO/ATENCION',
 u'OTROS',
 u'PRECIO',
 u'SITUACION',
 u'CONTENIDOS',
 u'NO_FIBRA']